<a href="https://colab.research.google.com/github/Retrich/git-practice/blob/main/Building_ANN_In_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# We make the artifical neural network!!

## And we have to recognize the pattern of building the model

### STAGE 1. Setting up the gpu environment

In [2]:
# install the library that we need
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8 MB 41 kB/s 
     |████████████████████████████████| 449 kB 44.9 MB/s 
     |████████████████████████████████| 3.8 MB 35.8 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=d006389a5b14e4e811e8bc4f4a4b24022c87743095cc0702fd62974e5c174ddb
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succes

In [3]:
# and we can import dependencies
import tensorflow as tf 
from tensorflow.keras.datasets import fashion_mnist # 당분간 텐서플로우 내의 데이터를 활용
import datetime
import numpy as np

In [4]:
print(tf.__version__)  # check the version

2.0.0


### STEP 2. Dataset preprocessing

In [11]:
# 훈련샘플과 검증샘플을 따로 저장, 강화학습이니까 x와 y를 모두 받아야 한다.
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()

# 그리고 데이터들의 벡터값이 0-1에 분포하도록 표준화를 진행해준다.
X_train = X_train / 255.0
X_test = X_test / 255.0

In [13]:
# 행렬의 계산을 위해 차원을 맞춰줘야한다.
# -1은 자동으로 맞춰지는 인수로, 28*28에 맞춰서 인수로 나온다.
# 열을 28*28개 가진 n개의 샘플행들을 보여준다.
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
print(X_train.shape)  # 열의 개수에 맞춰진 행의 개수를 확인한다.

(60000, 784)


### STEP 3. Building an artificial neural network

In [15]:
# 모델을 정의해주기 전에, 인스턴스를 생성한다.
model = tf.keras.models.Sequential()

# 사실 입력층도 하나의 층이지만, 세지 않는 층으로 구분하는 게 좋다.
# 전처리에 사실상 입력층을 구성하기 떄문이다.

# adding a first fully-connected hidden layer
# input_size는 입력층의 열을 행으로 받는다. 그리고 열은 다음 층의 행의 개수로 나와야하기에 미정.
model.add(tf.keras.layers.Dense(units=128, activation="relu", input_shape = (784, )))

# adding a second layer with dropout
# 드롭아웃은 과적합을 방지하기 위해 특정 비율만큼 0으로 만드는 부분집합 샘플을 만드는 방법이다.
# 지나친 샘플들을 모두 고려하는 과적합된 선을 만드는 것을 막는다.
model.add(tf.keras.layers.Dropout(0.2))

# adding the output layer
# output layer에서는 굳이 input_shape를 지정 안해도 된다. 
# 첫 Layer에서만 지정해주고 나머지는 units로 자동으로 받나보다.
model.add(tf.keras.layers.Dense(units=10, activation="softmax"))

In [17]:
# compiling the model
# forward pass로 설계된 model을 컴파일하는 과정이다.

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## STEP 4. Training the model

In [18]:
# 5번 훈련을 시켜보자
model.fit(X_train, Y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 8s 127us/sample - loss: 0.5317 - sparse_categorical_accuracy: 0.8105
Epoch 2/5
60000/60000 [==============================] - 6s 105us/sample - loss: 0.4045 - sparse_categorical_accuracy: 0.8516
Epoch 3/5
60000/60000 [==============================] - 6s 102us/sample - loss: 0.3673 - sparse_categorical_accuracy: 0.8658
Epoch 4/5
60000/60000 [==============================] - 6s 102us/sample - loss: 0.3464 - sparse_categorical_accuracy: 0.8738
Epoch 5/5
60000/60000 [==============================] - 6s 102us/sample - loss: 0.3318 - sparse_categorical_accuracy: 0.8781


## STEP 5. Model evaluaion and predicion 


In [20]:
test_loss, test_accuracy = model.evaluate(X_test, Y_test)

10000/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [21]:
print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.8672999739646912


## STEP 6. Saving the model

In [22]:
# saving the architecture (topology) of the network

model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
  json_file.write(model_json)

In [23]:
# saving network weights
model.save_weights("fashion_model.h5")